In [20]:
import matplotlib. pyplot as plt
import keras
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Keras
from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

#preprocessing required
from urllib import request

## Preprocessing of images

In [81]:
train_path='Mask_Datasets\Train'
test_path='Mask_Datasets\Validation'


train_batches=ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_path,
        color_mode='grayscale',
        target_size=(150, 150),
        shuffle=True,
        classes=['Mask','No_mask'])
            
test_batches=ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_path,
        target_size=(150, 150),
        shuffle=True,
        color_mode='grayscale',
        classes=['Mask','No_mask'])            
            

Found 750 images belonging to 2 classes.
Found 350 images belonging to 2 classes.


# Keras Model

In [82]:
epochs=30
width,height=150,150
num_features=64


model=Sequential()

model.add(Conv2D(num_features,(3,3),activation='relu',input_shape=(width,height,1)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(2*num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(2*num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(120,activation='relu'))
model.add(Dense(2,activation='softmax'))


model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 148, 148, 64)      640       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 15, 15, 128)      

In [83]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )

In [84]:
history=model.fit_generator(train_batches,epochs=25,
                           validation_data=test_batches,verbose=1,shuffle=True)

Epoch 1/25
24/24 [==============================] - 27s 1s/step - loss: 0.6940 - accuracy: 0.5467 - val_loss: 0.6379 - val_accuracy: 0.6029
Epoch 2/25
24/24 [==============================] - 24s 981ms/step - loss: 0.5994 - accuracy: 0.6800 - val_loss: 0.3848 - val_accuracy: 0.8343
Epoch 3/25
24/24 [==============================] - 24s 1s/step - loss: 0.4318 - accuracy: 0.8120 - val_loss: 0.2011 - val_accuracy: 0.8857
Epoch 4/25
24/24 [==============================] - 32s 1s/step - loss: 0.4250 - accuracy: 0.8147 - val_loss: 0.2159 - val_accuracy: 0.9057
Epoch 5/25
24/24 [==============================] - 26s 1s/step - loss: 0.3256 - accuracy: 0.8667 - val_loss: 0.1857 - val_accuracy: 0.9429
Epoch 6/25
24/24 [==============================] - 30s 1s/step - loss: 0.2418 - accuracy: 0.9147 - val_loss: 0.1172 - val_accuracy: 0.9400
Epoch 7/25
24/24 [==============================] - 29s 1s/step - loss: 0.2086 - accuracy: 0.9227 - val_loss: 0.1514 - val_accuracy: 0.9600
Epoch 8/25
24/24 

In [85]:
from keras.models import load_model

model.save('mask.h5')

In [63]:
model = load_model('mask.h5')


In [92]:
pred=model.predict(test_batches)

In [91]:
np.argmax(model.predict(test_batches[0][0])[0])

0

In [73]:
test_batches[0][0][0][0]

array([[0.38431376, 0.32156864, 0.23137257],
       [0.38431376, 0.32156864, 0.227451  ],
       [0.38431376, 0.32156864, 0.22352943],
       [0.38431376, 0.32156864, 0.22352943],
       [0.3803922 , 0.31764707, 0.21960786],
       [0.3803922 , 0.31764707, 0.21568629],
       [0.3803922 , 0.31764707, 0.21568629],
       [0.3803922 , 0.3137255 , 0.21568629],
       [0.38431376, 0.3137255 , 0.21960786],
       [0.38431376, 0.3137255 , 0.21960786],
       [0.38431376, 0.3137255 , 0.21960786],
       [0.38431376, 0.3137255 , 0.21960786],
       [0.38823533, 0.31764707, 0.22352943],
       [0.38823533, 0.31764707, 0.22352943],
       [0.38823533, 0.31764707, 0.22352943],
       [0.38823533, 0.31764707, 0.22352943],
       [0.38431376, 0.3137255 , 0.21960786],
       [0.38431376, 0.3137255 , 0.21960786],
       [0.38431376, 0.3137255 , 0.21960786],
       [0.38431376, 0.3137255 , 0.21960786],
       [0.38431376, 0.3137255 , 0.21960786],
       [0.38431376, 0.3137255 , 0.21960786],
       [0.

In [95]:
model.evaluate(pred,test_batches)

ValueError: Error when checking input: expected conv2d_33_input to have 4 dimensions, but got array with shape (350, 2)